In [51]:
import pandas as pd
import numpy as np

Задание 0. Определить вероятность выпадения цифры 8 на десятом броске 9-гранного кубика. Ответ укажите в комментариях в файле ipynb, где буду выполняться следующие задания.

---
 1/9 = 0.1111... (примерно 11.11%)
Поскольку броски независимы, вероятность выпадения конкретной цифры одинакова для любого броска

Задание 1.0: Импорт всех необходимых данных

In [52]:
file_path = 'Данные для задания Python.xlsx'

task1 = pd.read_excel(file_path, sheet_name='Задание1')

# Для отчетов о продажах нужно задать имена столбцов
column_names = [
    'Бренд', 'Предмет', 'Сезон', 'Коллекция', 'Наименование',
    'Номенклатура', 'Размер', 'Склад',
    'Поступления_шт', 'Поступления_себестоимость',
    'Заказано_шт', 'Заказано_себестоимость',
    'Выкупили_шт', 'Выкупили_руб', 'Текущий_остаток_шт'
]

# Читаем отчеты с пропуском первых двух строк, которые содержат заголовки
ivanov_sales = pd.read_excel(file_path, sheet_name='Отчет о продажах ИП Иванов', skiprows=2, header=None, names=column_names)
petrov_sales = pd.read_excel(file_path, sheet_name='Отчет о продажах ИП Петров', skiprows=2, header=None, names=column_names)
sidorov_sales = pd.read_excel(file_path, sheet_name='Отчет о продажах ИП Сидоров', skiprows=2, header=None, names=column_names)

cost_table = pd.read_excel(file_path, sheet_name='Себестоимость')
reference = pd.read_excel(file_path, sheet_name='Справочник')

In [53]:
print("\nЗадание 1")
print(task1.head())

print("\nОтчет Иванова")
print(ivanov_sales.head())

print("\nТаблица себестоимости")
print(cost_table)

print("\nСправочник категорий")
print(reference.head())


Задание 1
   Номенклатура  ИП  Заказы, шт.  Выручка, руб.  Прибыль, руб.  \
0      22740348 NaN          NaN            NaN            NaN   
1      27708426 NaN          NaN            NaN            NaN   
2      35217579 NaN          NaN            NaN            NaN   
3      74417961 NaN          NaN            NaN            NaN   
4      22740351 NaN          NaN            NaN            NaN   

   Рентабельность, %  
0                NaN  
1                NaN  
2                NaN  
3                NaN  
4                NaN  

Отчет Иванова
        Бренд              Предмет Сезон Коллекция  \
0  IvanovCase  Чехлы для телефонов   NaN       NaN   
1  IvanovCase  Чехлы для телефонов   NaN       NaN   
2  IvanovCase  Чехлы для телефонов   NaN       NaN   
3  IvanovCase  Чехлы для телефонов   NaN       NaN   
4  IvanovCase  Чехлы для телефонов   NaN       NaN   

                                        Наименование  Номенклатура  Размер  \
0  Силиконовый чехол на Айфон 4 / Ай

Задание 1.1: Определить к какому ИП относится номенклатура

In [54]:
# Задание 1.1 - Определяем к какому ИП относится номенклатура
def get_ip_for_nomenclature(nomenclature):
    in_ivanov = nomenclature in ivanov_sales['Номенклатура'].values
    in_petrov = nomenclature in petrov_sales['Номенклатура'].values
    in_sidorov = nomenclature in sidorov_sales['Номенклатура'].values

    if in_ivanov and not in_petrov and not in_sidorov:
        return 'Иванов'
    elif not in_ivanov and in_petrov and not in_sidorov:
        return 'Петров'
    elif not in_ivanov and not in_petrov and in_sidorov:
        return 'Сидоров'
    elif in_ivanov or in_petrov or in_sidorov:
        # Если номенклатура встречается у нескольких ИП, берем первого
        if in_ivanov:
            return 'Иванов'
        elif in_petrov:
            return 'Петров'
        else:
            return 'Сидоров'
    else:
        return '-'

task1['ИП'] = task1['Номенклатура'].apply(get_ip_for_nomenclature)


In [55]:
print(task1[['Номенклатура', 'ИП']].head(10))

   Номенклатура      ИП
0      22740348  Иванов
1      27708426  Иванов
2      35217579  Иванов
3      74417961  Петров
4      22740351  Иванов
5      23515101  Иванов
6      30393654  Иванов
7      23828211  Иванов
8      22740462  Иванов
9      24568431  Иванов


Задание 1.2: Посчитать количество заказов по каждой номенклатуре

In [56]:
def get_orders_for_nomenclature(nomenclature):
    orders = 0

    # Суммируем заказы из отчета Иванова
    if nomenclature in ivanov_sales['Номенклатура'].values:
        orders += ivanov_sales[ivanov_sales['Номенклатура'] == nomenclature]['Заказано_шт'].sum()

    # Суммируем заказы из отчета Петрова
    if nomenclature in petrov_sales['Номенклатура'].values:
        orders += petrov_sales[petrov_sales['Номенклатура'] == nomenclature]['Заказано_шт'].sum()

    # Суммируем заказы из отчета Сидорова
    if nomenclature in sidorov_sales['Номенклатура'].values:
        orders += sidorov_sales[sidorov_sales['Номенклатура'] == nomenclature]['Заказано_шт'].sum()

    return orders

task1['Заказы, шт.'] = task1['Номенклатура'].apply(get_orders_for_nomenclature)


In [57]:
print(task1[['Номенклатура', 'ИП', 'Заказы, шт.']].head(10))

   Номенклатура      ИП  Заказы, шт.
0      22740348  Иванов            6
1      27708426  Иванов            3
2      35217579  Иванов            5
3      74417961  Петров            6
4      22740351  Иванов           15
5      23515101  Иванов            3
6      30393654  Иванов           26
7      23828211  Иванов            4
8      22740462  Иванов           19
9      24568431  Иванов           10


Задание 1.3: Посчитать выручку по каждой номенклатуре

In [58]:
def get_revenue_for_nomenclature(nomenclature):
    revenue = 0

    # "Заказано, себестоимость" составляет 83% от выручки, поэтому делим на 0.83
    if nomenclature in ivanov_sales['Номенклатура'].values:
        revenue += ivanov_sales[ivanov_sales['Номенклатура'] == nomenclature]['Заказано_себестоимость'].sum() / 0.83

    if nomenclature in petrov_sales['Номенклатура'].values:
        revenue += petrov_sales[petrov_sales['Номенклатура'] == nomenclature]['Заказано_себестоимость'].sum() / 0.83

    if nomenclature in sidorov_sales['Номенклатура'].values:
        revenue += sidorov_sales[sidorov_sales['Номенклатура'] == nomenclature]['Заказано_себестоимость'].sum() / 0.83

    return round(revenue, 2)

task1['Выручка, руб.'] = task1['Номенклатура'].apply(get_revenue_for_nomenclature)


In [59]:
print(task1[['Номенклатура', 'ИП', 'Заказы, шт.', 'Выручка, руб.']].head(10))

   Номенклатура      ИП  Заказы, шт.  Выручка, руб.
0      22740348  Иванов            6        1168.77
1      27708426  Иванов            3         486.98
2      35217579  Иванов            5         865.72
3      74417961  Петров            6        1165.16
4      22740351  Иванов           15        2921.93
5      23515101  Иванов            3         459.90
6      30393654  Иванов           26        5627.28
7      23828211  Иванов            4         735.86
8      22740462  Иванов           19        3701.11
9      24568431  Иванов           10        1947.95


Задание 1.4: Посчитать прибыль по каждой номенклатуре

In [60]:
# Задание 1.4 - Считаем прибыль по каждой номенклатуре
# Находим минимальные фиксированные затраты
min_fixed_cost = cost_table['Фиксированные затраты, руб./шт.'].min()

# Создаем словари для категорий и затрат
fixed_costs_dict = dict(zip(cost_table['Категория'], cost_table['Фиксированные затраты, руб./шт.']))
category_dict = dict(zip(reference['Номенклатура'], reference['Категория']))

def calculate_profit(row):
    ip = row['ИП']
    revenue = row['Выручка, руб.']
    orders = row['Заказы, шт.']
    nomenclature = row['Номенклатура']

    if revenue == 0:
        return 0

    # Определяем налоговую ставку в зависимости от ИП
    if ip == 'Иванов':
        tax_rate = 0.01
    elif ip == 'Петров':
        tax_rate = 0.03
    elif ip == 'Сидоров':
        tax_rate = 0.05
    else:
        tax_rate = 0

    # Определяем категорию товара и фиксированные затраты
    category = category_dict.get(nomenclature, None)
    if category:
        fixed_cost = fixed_costs_dict.get(category, min_fixed_cost)
    else:
        fixed_cost = min_fixed_cost

    # Рассчитываем затраты
    commission = 0.17 * revenue  # Комиссия 17% от выручки
    tax = tax_rate * revenue  # Налог в зависимости от ИП
    fixed_costs = fixed_cost * orders  # Фиксированные затраты

    total_costs = commission + tax + fixed_costs
    profit = revenue - total_costs

    return round(profit, 2)

task1['Прибыль, руб.'] = task1.apply(calculate_profit, axis=1)

In [61]:
# Отображаем результаты задания 1.4
print("\nЗадание 1.4 - Прибыль по номенклатурам:")
print(task1[['Номенклатура', 'ИП', 'Заказы, шт.', 'Выручка, руб.', 'Прибыль, руб.']].head(10))


Задание 1.4 - Прибыль по номенклатурам:
   Номенклатура      ИП  Заказы, шт.  Выручка, руб.  Прибыль, руб.
0      22740348  Иванов            6        1168.77          37.99
1      27708426  Иванов            3         486.98         -60.88
2      35217579  Иванов            5         865.72         -57.11
3      74417961  Петров            6        1165.16          11.73
4      22740351  Иванов           15        2921.93          94.98
5      23515101  Иванов            3         459.90         -83.08
6      30393654  Иванов           26        5627.28         625.97
7      23828211  Иванов            4         735.86         -10.19
8      22740462  Иванов           19        3701.11         120.31
9      24568431  Иванов           10        1947.95         247.32


Задание 1.5: Посчитать рентабельность продаж по каждой номенклатуре

In [62]:
# Задание 1.5 - Считаем рентабельность продаж
def calculate_profitability(row):
    if row['Выручка, руб.'] == 0:
        return 0
    return round((row['Прибыль, руб.'] / row['Выручка, руб.']) * 100, 2)

task1['Рентабельность, %'] = task1.apply(calculate_profitability, axis=1)


In [63]:
print(task1.head(10))

   Номенклатура      ИП  Заказы, шт.  Выручка, руб.  Прибыль, руб.  \
0      22740348  Иванов            6        1168.77          37.99   
1      27708426  Иванов            3         486.98         -60.88   
2      35217579  Иванов            5         865.72         -57.11   
3      74417961  Петров            6        1165.16          11.73   
4      22740351  Иванов           15        2921.93          94.98   
5      23515101  Иванов            3         459.90         -83.08   
6      30393654  Иванов           26        5627.28         625.97   
7      23828211  Иванов            4         735.86         -10.19   
8      22740462  Иванов           19        3701.11         120.31   
9      24568431  Иванов           10        1947.95         247.32   

   Рентабельность, %  
0               3.25  
1             -12.50  
2              -6.60  
3               1.01  
4               3.25  
5             -18.06  
6              11.12  
7              -1.38  
8               3.25  

Задание 1.6: Выполнить экспорт получившейся таблицы в формате xlsx

In [64]:
output_path = 'Задание 1.xlsx'
task1.to_excel(output_path, sheet_name='Таблица', index=False)


Задание 1.7: Построить сводную таблицу

In [65]:
pivot_table = pd.pivot_table(
    task1,
    index='ИП',
    values=['Заказы, шт.', 'Выручка, руб.', 'Прибыль, руб.'],
    aggfunc='sum'
)

print("Сводная таблица:")
print(pivot_table)

Сводная таблица:
         Выручка, руб.  Заказы, шт.  Прибыль, руб.
ИП                                                
-                 0.00            0           0.00
Иванов       238766.31         1078       10048.72
Петров       119762.71          441        7679.92
Сидоров        3185.27           15          64.10
